# Processamneto das imagens histológicas de cérebros de rato

##### Import das bibliotecas necessárias

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

from skimage import io
from skimage import color
from skimage import morphology
from skimage import exposure
from skimage import measure
import cv2

### Pré-Processamento da Imagem Histológica

Deve colocar na variável `img_path` o caminho da imagem que pretende ser processada e em `output_path` o path da pasta onde pretende que sejam guardadas as imagens resultantes das operações

In [ ]:
output_path = r"C:\Users\vieir\Desktop\OUTPUT"

img_path = r"C:\Users\vieir\Desktop\Tese\Imagens\Slide1_10xmosaic_slice2.tif"
img = io.imread(img_path)

##### Aplicar um contraste maior na imagem

In [ ]:
arr = np.array([2, 50, 100, 205, 250], dtype=np.uint8)
min, max = arr.min(), arr.max()
arr_subtracted = arr - min
arr_divided = arr_subtracted / (max - min) 
arr_multiplied = arr_divided * 255 
arr_rescaled = np.asarray(arr_multiplied, dtype=arr.dtype)
percentiles = np.percentile(img, (0.5, 99.5))
scaled=exposure.rescale_intensity(img, in_range=tuple(percentiles))

fig, ax_arr = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 10))
ax1, ax2 = ax_arr.ravel()

ax1.imshow(img)
ax1.set_title('Original image')

ax2.imshow(scaled, cmap='gray')
ax2.set_title('Rescaled/Contrasted')

for ax in ax_arr.ravel():
    ax.set_axis_off()

plt.tight_layout()
plt.show()

`(Opcional) Correr a próxima célula para guardar a imagem obtida pelo processo anterior`

In [ ]:
img_name = os.path.splitext(os.path.basename(img_path))[0]
output_img = output_path + "\\" + img_name + "_Contrasted.tif"
io.imsave(output_img,scaled)

##### Criação da mascara da imagem

In [ ]:
lum = color.rgb2gray(scaled)
mask = morphology.remove_small_holes(
    morphology.remove_small_objects(
        lum < 0.80, 500),
    500)

mask = morphology.opening(mask)

`(Opcional) Correr a próxima célula para guardar a imagem obtida pelo processo anterior`

In [ ]:
output_img = output_path + "\\" + img_name + "_Mask.tif"
io.imsave(output_img,mask)

##### Isolamento da máscara da estrutura

In [ ]:
labeled_mask, num_labels = measure.label(mask, return_num=True)

# Get the sizes of all connected components
sizes = [np.sum(labeled_mask == label) for label in range(1, num_labels + 1)]

# Find the label corresponding to the largest connected component
largest_label = np.argmax(sizes) + 1  # Add 1 to account for 0-label (background)

# Create a new mask containing only the largest connected component
mask = labeled_mask == largest_label

# Display result
fig, ax_arr = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 10))
ax1, ax2 = ax_arr.ravel()

ax1.imshow(scaled)
ax1.set_title('Contrasted image')

ax2.imshow(mask, cmap='gray')
ax2.set_title('Mask')

for ax in ax_arr.ravel():
    ax.set_axis_off()

plt.tight_layout()
plt.show()

`Correr a próxima célula para guardar a imagem obtida pelo processo anterior`

In [ ]:
output_img = output_path + "\\" + img_name + "_Final_Mask.tif"
io.imsave(output_img,mask)

##### Aplicação da máscara na imagem original/com contraste para remoção do fundo

Escolha a imagem que pretende aplicar a máscara

* `imagem = img` para a imagem original
* `imagem = scaled` para a imagem original após contraste

In [ ]:
imagem = img

removed_background = imagem.copy()
removed_background[mask == 0] = 0

# Display the results
fig, axes = plt.subplots(1, 2)
axes[0].imshow(img)
axes[0].set_title('Image')
axes[1].imshow(removed_background)
axes[1].set_title('Image without Background')
for ax in axes:
    ax.axis('off')
plt.show()

`(Opcional) Correr a próxima célula para guardar a imagem obtida pelo processo anterior`

In [ ]:
img_name = os.path.splitext(os.path.basename(img_path))[0]
output_img = output_path + "\\" + img_name + "_Processed.tif"
io.imsave(output_img,removed_background)

### Criação das Macros para utilização no ImageJ

Neste ponto será necessário relacionar a imagem do atlas correspondente à imagem que utilizou até este ponto, colocando a máscara da imagem do atlas em `atlas_mask` e a máscara produzida e guardada anteriormente em `image_mask`.

Adicionalmente poderá adicionar a pasta onde deseja guardar o output das operações em `output_path`.

As máscaras das imagens do atlas encontram-se no repositório em que este documento se encontra

In [ ]:
atlas_mask_path = r"C:\Users\vieir\Desktop\Tese\Imagens\RBSC5_Blue_035_Masc_small.tif"
image_mask_path = r"C:\Users\vieir\Desktop\Tese\Imagens\Slide1_10xmosaic_slice2_Final_Mask.tif"

atlas_mask_name = os.path.basename(atlas_mask_path)
image_mask_name = os.path.basename(image_mask_path)

atlas_mask = io.imread(atlas_mask_path)
image_mask = io.imread(image_mask_path)

##### Criação da Macro para obter os coeficientes de transformação

In [ ]:
ratio = image_mask.shape[1]/atlas_mask.shape[1]

new_img_height = int(image_mask.shape[0]/ratio)
new_img_width = int(image_mask.shape[1]/ratio)

In [ ]:
first_macro_path = output_path + "\Macro1.ijm"

content = [
    'open("' + image_mask_path.replace("\\", "/") + '");',
    'run("Size...", "width=' + str(new_img_width) + ' height=' + str(new_img_height) + ' depth=1 constrain average interpolation=Bilinear");',
    'open("' + atlas_mask_path.replace("\\", "/") + '");',
    'run("bUnwarpJ", "source_image=' + atlas_mask_name + ' target_image=' + image_mask_name + ' registration=Mono image_subsample_factor=0 initial_deformation=[Very Coarse] final_deformation=Fine divergence_weight=0 curl_weight=0 landmark_weight=0 image_weight=1 consistency_weight=10 stop_threshold=0.01 save_transformations save_direct_transformation=' + output_path.replace("\\", "/") + '/Coeficientes.txt");',
    'close();',
    'close();',
    'close();'
]

with open(first_macro_path, "w") as file:
    file.writelines("\n".join(content))

##### Criação da Macro para aplicação dos coeficientes na imagem original do atlas

O Path da imagem do atlas correspondente ao processamento anterior deverá ser colocado em `atlas_path`.

!Atenção! Se o nome ou o local predefinidos do ficheiro dos coeficientes da transformação obtido anteirormente for modificado, será necessário apontar as alterações em `coeficients_file_path`.

In [ ]:
atlas_path = r"C:\Users\vieir\Desktop\Tese\Imagens\TIFFAtlasBlueLabel\RBSC5_Blue_035.tif"
atlas_name = os.path.basename(atlas_path)

coeficients_file_path = output_path + "\Coeficientes.txt"

new_atlas_height = int(atlas_mask.shape[0]*ratio)
new_atlas_width = int(atlas_mask.shape[1]*ratio)

In [ ]:
second_macro_path = output_path + "\Macro2.ijm"

content = [
    'open("' + atlas_mask_path.replace("\\", "/") + '");',
    'open("' + atlas_path.replace("\\", "/") + '");',
    'call("bunwarpj.bUnwarpJ_.loadElasticTransform", "' + coeficients_file_path.replace("\\","\\\\") + '", "' + atlas_mask_name + '", "' + atlas_name + '");',
    'run("Size...", "width=' + str(new_atlas_width) + ' height=' + str(new_atlas_height) + ' depth=1 constrain average interpolation=Bilinear");',
    'saveAs("Tiff", "' + output_path.replace("\\", "/") + '/AtlasTransformed.tif");',
    'close();',
    'close();'
]

with open(second_macro_path, "w") as file:
    file.writelines("\n".join(content))

### Sobreposição das imagens

As duas imagens a ser sobrepostas por defeito são a imagem original e a imagem gerada na etapa anterior.

Se quiser alterar qualquer uma destas deverá colocar o path das mesmas em `img_path` e `atlas_path`.

Ainda deverá alterar o valor de `atlas_path` se mover ou alterar o nome a imagem produzida na etapa anterior.

In [ ]:
img_path = img_path
atlas_path = output_path + "\AtlasTransformed.tif"

img = cv2.imread(img_path)
atlas = cv2.imread(atlas_path)

atlas = cv2.cvtColor(atlas, cv2.COLOR_BGR2RGB)

In [ ]:
height1, width1 = img.shape[0],img.shape[1]
print("Image:",height1, width1)

height2, width2 = atlas.shape[0],atlas.shape[1]
print("Atlas:",height2, width2)

Nota: Quando mais díspares forem os valores entre as alturas das imagens piores serão os resultados obtidos desta sobreposição das imagens. 

Para corrigir este erro as imagens iniciais do projeto devem ser editadas de forma a conterem ratios semelhantes.

In [ ]:
min_height = np.minimum(height1, height2)
min_width = np.minimum(width1, width2)

img11 = img[0:min_height, 0:min_width]
img22 = atlas[0:min_height, 0:min_width]

img_add = cv2.addWeighted(img11, 0.4, img22, 0.6, 0) 
plt.imshow(img_add)
plt.show()

`Correr a próxima célula para guardar o resultado final deste processo`

In [ ]:
cv2.imwrite(output_path + '\Resultado.tif', img_add)